## Classification with Audio

You were introduced to classification and clustering in the last section, and to STFT in the last lab.
All of these concepts will be used in this week's lab. 

This lab has two components: 
- Part 1: Audio-based classification for detecting voice (here)
- Part 2: Image segmentation based on pixel values and location (please see other ipynb)

### Necessary packages

- scipy
- matplotlib
- numpy
- IPython

If you are using Canopy or Anaconda, you wouldn't need any additional packages for this lab. If needed, you can install packages using `pip install` or `conda install` depending on your environment.



In [ ]:
import numpy as np
import math
from __future__ import print_function
from matplotlib import pyplot as plt
from scipy.io import wavfile
from scipy.fftpack import dct
from IPython.display import Audio
from collections import defaultdict

%matplotlib inline

# Part 1: Mel Frequency Cepstral Coefficients 

You were introduced to the short time fourier transform in the last lab. The basic idea is to divide the audio into sections, apply appropriate windowing functions and then visualize the frequency content variation across time. In this week's lab we will use the MFCC coefficients:

The **Mel Frequency Cepstrum** is inspired by the human auditory response. It is observed that the human ear acts as a collection of filters, and concentrates only on certain frequency components. These filters are non-uniformly spaced on the frequency axis.

![](melfilterbank.jpg)

The *Mel Frequency Cepstrum* audio feature is being used succesfully in many audio processing applications. We will need some math tools from scipy to implement this audio feature extractor.

The Mel Frequency Cepstral Coefficients (MFCC) are computed as follows:

1. Compute the Fourier Transform of the (windowed) signal
2. Map power spectrum to the mel scale, using overlapping triangular windows (filter banks) as above. 
3. Take log of the power spectrum in the mel 'frequency' (one value corresponding to each mel filter)
4. Compute the Discrete Cosine Transform of the spectrum computed above, as though it were a signal

The amplitudes obtained are the MFCC. A block diagram of the process is shown here:
![](MFCC.png)

The reason for Step 4 above is because the filter bank coefficients computed in the previous step are highly correlated, which could be problematic in some machine learning algorithms. The application of transforms like the Discrete Cosine Transform (DCT) decorrelate the filter bank coefficients and yield a compressed representation of the filter banks. 

Typically, for Automatic Speech Recognition (ASR), the resulting cepstral coefficients 2-13 are retained and the rest are discarded; The reasons for discarding the other coefficients is that they represent fast changes in the filter bank coefficients and these fine details don’t contribute to Automatic Speech Recognition (ASR).

We implement each component below.

### Loading the wav data and hearing + visualizing the signal 

In [ ]:
audio_file = 'data/italian.wav'
Fs, signal = wavfile.read(audio_file)

highfreq = Fs/2 # Nyquist rate
lowfreq = 0    

print('Low Frequency', lowfreq, 'High Frequency', highfreq)

In [ ]:
Audio(audio_file)

In [ ]:
print('Shape:', np.shape(signal))

In [ ]:
plt.figure()
plt.plot(signal)
plt.title('Italian speech')
plt.show()

## <span style="color:blue">Exercise 1:  </span>
Write a function to generate the power spectrum of a signal:

$$ P = \frac{|\mathcal{F}(x)|^2}{N} $$ where $N$ is the number of points in the Fourier transform $\mathcal{F}$. This will be used in the final exercises.

In [ ]:
def get_power_spectrum(signal, NFFT):
    ''' Function to compute the power spectrum of a signal 
        using the formula above
        Args: 
            signal: [ndarray] audio signal
            NFFT: [int] number of points in the Fourier transform
        Returns:
            power: [ndarray] the corresponding power spectrum
        Hint:
            - You can use rfft from numpy.fft as in the last lab
    '''
    
    power =  
    return power

### Converting from frequency to mel frequency and vice versa

The formula to convert frequency $f$ Hertz to $m$ Mel is:

$$m = 2595 * log_{10}(1+\frac{f}{700})$$

## <span style="color:blue"> Exercise 2</span>

1. Derive the formula to convert from Mel back to Hertz
2. Implement these 2 conversions as functions

The formula to convert Mel back to Hertz is:

Your answer here:

In [ ]:
def hz2mel(freq):
    ''' This function should output the Mel frequencies given some input frequency
        h is a 1-d array of input frequencies in hertz
        Args:
            freq: [ndarray] frequencies in hertz
        Returns:
            mels: [ndarray] Mel scale frequncies based on above formula
    '''
    ## Your code here
    return mels

def mel2hz(mel):
    ''' This function should output the frequency in Hertz given some input quefrency
        m is a 1-d array of input quefencies in Mel
        Args:
            mels: [ndarray] Mel scale frequncies 
        Returns:
            freq: [ndarray] frequencies in hertz based on above formula          
    '''
    ## Your code here
    return freq

## Defining the Mel-spaced filter banks

The Mel-spaced filterbank is the set of triangular window filter which corresponds to human ear hearing perception.
![](melfilterbank.jpg)
The following code is to use to generate **num_filters** filters using the end points supplied by **bin_points** vector. You do not need to edit this part.

In [ ]:
#filter bank generation
def filter_bank(num_filters, NFFT, bin_points):
    ''' Generate num_filters filters using the
        end points supplied by bin_points vector
        Args:
            num_filters: [int] number of Mel filters
            NFFT: [int] number of points in the FFT
            bin_points: end points of the Mel filters
        Returns:
            fbank: [ndarray] filter bank as visualized above
    '''
    fbank = np.zeros([num_filters,int(NFFT/2+1)])
    for j in range(0,num_filters):
        for i in range(int(bin_points[j]),int(bin_points[j+1])):
            #rising edge of the triangle
            fbank[j,i] = (i - bin_points[j])/(bin_points[j+1]-bin_points[j]) 
            
        for i in range(int(bin_points[j+1]),int(bin_points[j+2])):
            #falling edge of the triangle
            fbank[j,i] = (bin_points[j+2]-i)/(bin_points[j+2]-bin_points[j+1]) 
    return fbank

Let's now look at how the Mel-spaced filter banks are applied to the power spectrum of an audio signal. The following code generates the filter banks given the sampling frequency, number of filters, and length of the FFT.(You will need to finish the code for previous section for this part to run)

In [ ]:
lowmel = hz2mel(lowfreq)     # convert to Mel scales
highmel = hz2mel(highfreq)

NFFT = 512            # number of FFT points
num_filters = 20            # number of Mel spaced filter banks

# convert the frequency into Mel points
mel_points = np.linspace(lowmel, highmel, num_filters + 2) # linear in Mel scale correspondings to human perception of sound
bin_points = np.floor(mel2hz(mel_points)/Fs*(NFFT+1)) # convert back to Hert 

fbank = filter_bank(num_filters, NFFT, bin_points)

In [ ]:
mel_points

Here's a visualization of the 20 filter banks in the frequency domain

In [ ]:
freq = np.arange(257)*(Fs/2.)/257    # generate frequencies for plot
plt.figure(figsize = (12, 9)) # make a bigger figure
for i in np.arange(num_filters):
    plt.plot(freq, fbank[i,:]) # plot 20 filter banks
plt.title('Filter Bank Responses')
ax = plt.axes()
ax.set_xlabel('frequency')
ax.set_ylabel('amplitude')
ax.set_xlim([0, 22050]);

In [ ]:
print(bin_points)

### Question: looking at the filter bank, are human hearing sensitive to low or high frequencies?

Your answer here:

## <span style="color:blue"> Exercise 3</span>

Use the code from your previous lab to generate and plot a spectogram for the given wave segment `wav_segment`. The variable `data` should contain the spectrogram values. 

In [ ]:
fig, (ax1) = plt.subplots(ncols=1) # create plot
fig.set_size_inches(15, 5)

N=512
M=128

wav_segment = signal[200000:411000]

# generate & plot spectrogram (built-in function)

# Your code here:

## <span style="color:blue"> Exercise 4</span>

To generate the MFCC, we first need to filter the frequency content through the Mel filter bank `fbank` we have created. 
To do so, use the `dot()` function from *numpy* to multiply the `fbank` and `data` and get the filter coefficients `mel_filtered`. Also, take the `log` of the filter coefficients and store in `log_filtered`.

In [ ]:
mel_filtered = 
log_filtered = 

We can visualize the filter bank coefficients `mel_filtered` using the code below:

In [ ]:
plt.figure(figsize=(12,6))
plt.imshow(np.repeat(mel_filtered[::-1, :], 40, axis=0), cmap='jet', interpolation='nearest')
plt.colorbar()
plt.show()

### Generating the MFCCs

The Mel Frequency Cepstral Coefficients (MFCCs) are the Discrete Cosine Transform **[DCT](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.fftpack.dct.html)** of the log coefficients `log_filtered`. 


We can visualize the MFCCs for a particular segment of the audio clip as below:

In [ ]:
mfcc = dct(log_filtered[:, 5].T, type = 2, axis = 0, norm = 'ortho')# take the Discrete Cosine Transform

print(mfcc)

plt.figure()
plt.plot(mfcc)
plt.title('MFCC of window of "italian.wav"')
ax = plt.axes()
ax.set_xlabel('frequency bin')
ax.set_ylabel('value')

## <span style="color:blue"> Exercise 5</span>


With the information and code examples provided above, you should now be able to implement a function to compute the MFCC of a given signal. 

In [ ]:
def calculate_MFCC(Fs, data, N, M):
    ''' Function to compute MFCC by dividing the signal into
        frames/sections of length N, and obtaining the MFCC of that frame.
        Args:
            Fs: [int] sampling frequency
            data: [array] signal to compute MFCC
            N: [int] length of section to use
            M: [int] amount of overlap in consecutive sections
        Returns:
            mfcc: [number_of_frames x num_Mel_filters] MFCC coefficients for each section
                  where you will compute number_of_frames such that each frame is of length N and
                  has consecutive frames have an overlap of M
                  and num_Mel_filters is the number of filters in the Mel filter bank
    '''
    
    #Your code here
    # 1. Compute number of frames/sections:
    # Hint: look at the file mfcclab.py to find the number of frames in a signal 
    
    numberOfFrames = 
    
    # 2. for each frame select the corresponding section of the data signal, i.e. from i*M to i*M + N for the i^th frame
    # - zero-pad the tail frames to be of size N
    # - compute the power spectrum using your function from Exercise 1
    # - dot product with the Mel filter bank
    # - take the log
    # - compute the DCT
    
    mfcc = 

    return mfcc

# Speaker Identification

In this exercise, we will utilize the MFCC feature to recognize the identity of a speaker. You should put the two folders that were provided, train and test, into the same folder of your ipython notebook. There are 8 training data files and 8 testing data files, s1.wav to s8.wav. The training and testing data are already correctly labeled, i.e, s1.wav in the training folder matches with s1.wav in the testing folder (same person speaking). Nearest neighbor will be used to label the testing data.

Each audio file in this dataset is mono-channel. However the files are of different length, which requires special attention to allocate the training dataset. For each audio file in the train folder, we create its MFCC matrix, where each row corresponds to 12 MFCC's of one frame. If your signal has 125 frames, for instance, the size of MFCC matrix will be 125 by 12. The MFCC matrix of all audio files are stacked into one matrix called the **codebook**. At the same time, we also store the labels of the training data. 

## Training

We first calculate the total number of frames of our whole dataset. This is used to initialize the codebook matrix and the label vector. Let's say you have only 2 audio files. The first file has 10 frames, and the second has 20 frames. Then the size of your **codebook** is 30 by 12. 

The length of your **labels** vector will be 30, in which the first 10 elements are *1* and the last 20 elements are *2*. For this exercise, we will choose the length of each frame to be 512 and the step size is one third of the frame length.

In [ ]:
import mfcclab
N = 512
M = int(512/3)
print(N, M)
sumNumFrames = mfcclab.get_num_frames("train", N, M) # total number of frames for all of the training data. 

We define the range of frequencies for our filter banks in Hertz, *lowfreq* and *highfreq* (from 0 Hz to the Nyquist rate). To create the endpoints for our filter banks, we transform this range into mel-scale range. The *for* loop loads all the training data in the **train** folder, computes the MFCC's, and appends them to the **codebook**. Since we know the labels of our data, this is considered an example of *supervised learning*, as opposed to an *unsupervised learning* paradigm, like K-means clustering, where the labels are unknown and must be learned.

In [ ]:
highfreq = Fs/2 # Nyquist rate
lowfreq = 0 # lower end of audio spectrum

lowmel = hz2mel(lowfreq) # convert to Mel scales
highmel = hz2mel(highfreq)

NFFT = 512 # number of FFT points

nfilt = 20 # number of Mel spaced filter banks

# convert the frequency into Mel points
melPoints = np.linspace(lowmel, highmel, nfilt + 2)
binPoints = np.floor(mel2hz(melPoints)/Fs*(NFFT+1))

numKeptCoeff = 12 # we only keep the lower 12 coefficients since the higher frequencies 
# do not help speaker identification. we also ignore index 0 since this corresponds to the 
# volume level / DC value

fbank = filter_bank(nfilt, NFFT, binPoints) # create the Mel-spaced filter bank
codeBook = np.empty((sumNumFrames, numKeptCoeff-1)) #preallocate the training data
labels = np.zeros((sumNumFrames,)) # and the labels

currentIdx = 0
for i in range(1,9): # we have 8 signals
    filename = "train/s%d.wav" % i # this is how you replace %d with i in Python 2. 
    fs, data = wavfile.read(filename)
    mfcc = calculate_MFCC(fs, data, N, M)[:,1:numKeptCoeff]
    codeBook[currentIdx:currentIdx + mfcc.shape[0],:] = mfcc# assign this training data to codebook
    labels[currentIdx:currentIdx + mfcc.shape[0]] = i # and label them
    currentIdx = currentIdx + mfcc.shape[0] # move to next block in codeBook and labels

Since the each codeword in the codebook is of dimension 11, it is not possible to visualize the distribution of the codewords and their proximity to each other. However, tools exist that attempt to reduce the dimensionality of data while preserving the relative distances between data points. The following code uses a tool called Isomap (a non-linear dimension-reduction technique, unlike PCA) to plot the data in 2D to give some intuition of the distribution of the codewords. 

### Question: How well grouped are the codewords for each speaker?

Your answer here:

In [ ]:
mfcclab.plot_codebook_2D(codeBook, labels, ["s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8"])

## <span style="color:blue"> Exercise 6</span>


1. Use your favourite classifier from part I of the course to train on the given data based on the MFCCs for each frame using the data in the `train` folder

In [ ]:
classifier = 
classifier.fit(codeBook, labels) 

2. Load a test file

In [ ]:
filename = "test/s6.wav" #try load one testing wave file
fs, data = wavfile.read(filename)
mfcc = calculate_MFCC(fs, data, N, M)

We will now predict the label of each frame you just computed. There will be some mislabeled data. 

In [ ]:
pred_label = np.round(classifier.predict(mfcc[:,1:numKeptCoeff]))
print(pred_label)

In [ ]:
 # this is used to find the most occured label
d = defaultdict(int)
for i in pred_label:
    d[i] += 1
result = max(d.items(), key=lambda x: x[1])
print("Matching speaker:", result) #the first result is the label,
# the second is the number of occurences

3. Try different speakers for testing above. 
    - Is any speaker wrongly classified? 
    - Which speaker has the lowest number of votes but is correctly classified?
    - How can this classification task be improved?